In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [74]:
caseAbstractions = pd.read_stata('baseData/caseAbstractions.dta')

/Users/burke/opt/anaconda3/lib/python3.7/site-packages/pandas/io/stata.py:370: UserWarning: Encountered %tC format. Leaving in Stata Internal Format.
  warnings.warn("Encountered %tC format. Leaving in Stata " "Internal Format.")


In [77]:
caseAbstractions['intubation_date'] = pd.to_datetime(caseAbstractions['intubation_date'], unit='ms')
caseAbstractions['shock_date'] = pd.to_datetime(caseAbstractions['shock_date'], unit='ms')

In [80]:
caseAbstractions = caseAbstractions.drop(caseAbstractions[caseAbstractions.outcome_reviewer.isnull()].index, axis='index')

In [81]:
caseAbstractions = caseAbstractions[['pat_enc_csn_id', 'mrn', 'ed_arrival_dttm', 'hypertension', 'diabetes', 'asthma',
       'copd', 'chronic_lung', 'home_o2', 'osa', 'immunocompromised', 'pregnant', 'intubation', 'intubation_date',
                                    'shock', 'shock_date', 'dni_dnar', 'death', 'death_date']]

In [89]:
caseAbstractions.head()

,pat_enc_csn_id,mrn,ed_arrival_dttm,hypertension,diabetes,asthma,copd,chronic_lung,home_o2,osa,...,pregnant,intubation,intubation_date,shock,shock_date,dni_dnar,death,death_date,primaryOutcome,primaryOutcomeDate
0,272071328,101224505,1898955387000,Yes,Yes,No,Yes,No,No,No,...,No,Did not require supplemental oxygen,NaT,No,NaT,No,No,NaT,False,NaT
1,272254374,100340985,1899367347000,Yes,No,Yes,No,Yes,Yes,No,...,No,Intubated,2030-03-13 13:40:27,Yes,2030-03-13 19:30:27,Yes,Yes,2020-03-30,True,2020-03-30 00:00:00
2,272270483,28708828,1899376887000,Yes,No,No,No,Yes,No,Yes,...,No,Did not require supplemental oxygen,NaT,No,NaT,No,No,NaT,False,NaT
3,272529195,23334525,1899710787000,Yes,Yes,Yes,No,No,No,No,...,No,Intubated,2030-03-20 13:40:27,No,NaT,No,No,NaT,True,2030-03-20 13:40:27
4,272580543,100437568,1899758487000,Yes,No,No,No,No,No,No,...,No,Did not require supplemental oxygen,NaT,No,NaT,No,No,NaT,False,NaT


In [82]:
caseAbstractions['primaryOutcome'] = (caseAbstractions['intubation'] == 'Intubated') | (caseAbstractions['shock'] == 'Yes') | (caseAbstractions['death'] == 'Yes')

In [87]:
caseAbstractions['primaryOutcomeDate'] = caseAbstractions[['intubation_date', 'shock_date', 'death_date']].min(axis='columns')

In [65]:
rawData = pd.read_csv('LAB orders Details for potential COVID19 Mar 01-24 2020.csv')

In [66]:
encounters = pd.read_csv('Lab Encounters for potential COVID19 Mar 01-29 2020.csv')
encounters = encounters[['PAT_ENC_CSN_ID', 'COVID19_TESTED', 'COVID19_POSTIVE', 'ISOLATION']]

In [69]:
rawData.columns

Index(['COVID19_PATIENT', 'DX_NAME', 'ISOLATION_x', 'PATIENT_LIST',
       'PAT_CLASS', 'COMPONENT_BASE_NAME', 'PAT_MRN_ID', 'PAT_ID',
       'HSP_ACCOUNT_ID', 'PAT_ENC_CSN_ID', 'ORDER_ID', 'DESCRIPTION',
       'COMPONENT_ID', 'ORDER_INST', 'RESULT_TIME', 'RESULT_LINE',
       'COMPONENT_NAME', 'ORD_VALUE', 'REFERENCE_LOW', 'REFERENCE_HIGH',
       'RESULT_FLAG', 'ABNORMAL_YN', 'PAT_AGE_AT_ENC', 'ENC_TYPE',
       'CONTACT_DATE', 'APPT_TIME', 'HOSP_ADMSN_TIME', 'HOSP_DISCHRG_TIME',
       'ADMIT_TYPE', 'ORDER_TYPE_C', 'ORDER_STATUS_C', 'RESULTS_CMT',
       'COVID19_TESTED', 'COVID19_POSTIVE', 'ISOLATION_y'],
      dtype='object')

In [68]:
rawData = rawData.merge(encounters,how='left', on='PAT_ENC_CSN_ID', validate='many_to_one')

In [78]:
rawData = rawData.loc[rawData['COVID19_POSTIVE'] == 'Y']

In [81]:
rawData.PAT_MRN_ID.nunique()

144

In [87]:
rawData = rawData[['PAT_MRN_ID', 'ADMIT_TYPE', 'COMPONENT_NAME', 'ORD_VALUE', 'PAT_AGE_AT_ENC', 'RESULT_TIME']]
rawData.replace('C-REACTIVE PROTEIN SCREEN', value='crp', inplace=True)
rawData.replace('FERRITIN', value='ferritin', inplace=True)
rawData.replace('D-DIMER (SOFT)', value='dDimer', inplace=True)
rawData.sort_values(by=['PAT_MRN_ID', 'COMPONENT_NAME', 'RESULT_TIME'], ascending=True, axis='index', inplace=True)
print(len(rawData))
rawData.drop_duplicates(subset=['PAT_MRN_ID','COMPONENT_NAME' ],keep='first', inplace=True)
print(len(rawData))

134
134


In [88]:
rawData = rawData.loc[rawData.COMPONENT_NAME.isin(['crp', 'ferritin', 'dDimer'])]

In [89]:
wide = rawData.pivot(index='PAT_MRN_ID', columns='COMPONENT_NAME', values='ORD_VALUE')

In [90]:
ages = rawData.groupby('PAT_MRN_ID').first().rename(columns={'PAT_AGE_AT_ENC':'age'})
cleanedData = pd.concat([wide,ages],1)
cleanedData.crp.replace('<0.2', 0.1, inplace=True)
cleanedData.dDimer.replace('<0.17', 0.1, inplace=True)
cleanedData.dDimer.replace('>35.00', 40, inplace=True)
cleanedData.ferritin.replace('>16500.0', 1650, inplace=True)
cleanedData.ferritin.replace('>1650.0', 1650, inplace=True)

cleanedData.crp = cleanedData.crp.astype('float')
cleanedData.dDimer = cleanedData.dDimer.astype('float')
cleanedData.ferritin = cleanedData.ferritin.astype('float')
cleanedData.drop(labels=['ADMIT_TYPE', 'COMPONENT_NAME', 'ORD_VALUE', 'RESULT_TIME'], axis='columns',inplace=True)

In [94]:
cleanedData.head()

,crp,dDimer,ferritin,age
PAT_MRN_ID,,,,
11938221,NaN,NaN,189.7,50
12105058,1.9,0.54,67.3,63
12810659,10.0,0.80,1451.5,81
13769217,4.3,0.88,1157.4,71
15547972,16.1,0.96,713.9,61


In [96]:
import statsmodels.imputation.mice as mice

def ols_formula(df, dependent_var):
    df_columns = list(df.columns.values)
    df_columns.remove(dependent_var)
    fml = ''
    for col in df_columns:
        fml = fml + ' + ' + col
    return fml

imputedData = mice.MICEData(cleanedData)

for var in ['age', 'crp', 'dDimer', 'ferritin']:
    imputedData.set_imputer(var, formula=ols_formula(cleanedData, var))

imputedData.update_all(20)

In [107]:
cleanedData = imputedData.data

In [108]:
import scipy.special as scipySpecial
simData = cleanedData

# need to do some imputation here...will start with something simple...
#simData['crp'].fillna(simData.crp.mean(), inplace=True)
#simData['dDimer'].fillna(simData.dDimer.mean(), inplace=True)
#simData['ferritin'].fillna(simData.ferritin.mean(), inplace=True)


simData['linPredictor'] = (cleanedData.crp-cleanedData.crp.mean())*1.2+(cleanedData.dDimer-cleanedData.dDimer.mean())*2+(cleanedData.age-cleanedData.age.mean())/10*4 + np.random.normal(10)
simData['linPredictorCentered'] = simData.linPredictor - simData.linPredictor.mean()

simData['probDead'] = scipySpecial.expit(simData.linPredictorCentered)
simData['dead'] = simData.probDead > 0.9
simData = simData[['crp', 'dDimer', 'ferritin', 'age', 'dead']]

In [109]:
simData.dead.value_counts()

False    36
True     17
Name: dead, dtype: int64

In [110]:
simData

,crp,dDimer,ferritin,age,dead
0,5.3,3.03,189.7,50,False
1,1.9,0.54,67.3,63,False
2,10.0,0.80,1451.5,81,True
3,4.3,0.88,1157.4,71,False
4,16.1,0.96,713.9,61,True
5,17.2,4.33,97.8,82,True
6,6.0,0.59,1490.4,40,False
7,6.0,0.62,189.7,72,False
8,6.2,0.46,58.7,37,False
9,30.0,2.72,223.7,86,True


In [118]:
import pymc3 as pm

dead = simData['dead']
crp = simData.crp - simData.crp.mean()
ferritin = simData.ferritin - simData.ferritin.mean()
dDimer = simData.dDimer - simData.dDimer.mean()
age = simData.age - simData.age.mean()


with pm.Model() as model_simple:
    alpha = pm.Normal('alpha', mu=0, sd=10)
    betaCRP = pm.Normal('betaCRP', mu=0, sd=10)
    betaFerritin = pm.Normal('betaFerritin', mu=0, sd=10)
    betaDDimer = pm.Normal('betaDDimer', mu=0, sd=10)
    betaAge = pm.Normal('betaAge', mu=0.095, sd=0.05)

    
    mu = alpha + betaCRP * crp + betaDDimer * dDimer + betaAge * age + betaFerritin*ferritin   
    θ = pm.Deterministic('θ', pm.math.sigmoid(mu))
    
    y_1 = pm.Bernoulli('y_1', p=θ, observed=dead)

    trace_simple = pm.sample(1000, tune=1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [betaAge, betaDDimer, betaFerritin, betaCRP, alpha]
Sampling 4 chains, 0 divergences: 100%|██████████| 8000/8000 [00:06<00:00, 1257.25draws/s]


In [117]:
noPriors = trace_simple

In [120]:
az.summary(noPriors, var_names=['alpha', 'betaCRP', 'betaFerritin', 'betaDDimer', 'betaAge'])

,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
alpha,-12.641,6.364,-25.029,-1.396,0.227,0.164,785.0,757.0,810.0,1223.0,1.01
betaCRP,6.812,2.498,2.246,11.290,0.119,0.084,443.0,443.0,439.0,1038.0,1.01
betaFerritin,-0.002,0.006,-0.014,0.010,0.000,0.000,870.0,870.0,880.0,868.0,1.01
betaDDimer,13.636,6.020,3.663,24.551,0.274,0.194,481.0,481.0,480.0,937.0,1.01
betaAge,2.210,0.786,0.776,3.636,0.037,0.026,449.0,449.0,444.0,901.0,1.01


In [121]:
az.summary(trace_simple, var_names=['alpha', 'betaCRP', 'betaFerritin', 'betaDDimer', 'betaAge'])

,mean,sd,hpd_3%,hpd_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
alpha,-1.390,1.012,-3.283,0.492,0.027,0.020,1449.0,1287.0,1462.0,1534.0,1.0
betaCRP,0.700,0.224,0.292,1.112,0.006,0.004,1521.0,1448.0,1568.0,1952.0,1.0
betaFerritin,0.000,0.001,-0.002,0.002,0.000,0.000,1936.0,1507.0,1947.0,1943.0,1.0
betaDDimer,1.932,0.968,0.455,3.677,0.026,0.019,1439.0,1348.0,1573.0,1817.0,1.0
betaAge,0.172,0.041,0.096,0.249,0.001,0.001,1815.0,1800.0,1808.0,2025.0,1.0
